<a href="https://colab.research.google.com/github/kerupinen/Compdb_final_repo/blob/main/Final_Project_Rupinen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installing and Connecting to Database

In [95]:
pip install pymongo

In [96]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient(host='3.230.203.12',  # host is the hostname for the database
                                port=27018,  # port is the port number that mongo is running on
                                username='compdb_student',  # username for the db
                                password='compdbpass',  # password for the db
                                authSource='admin')  # Since our user only exists for the emse6992 db, we need to specify this

# Setup collections and connections
db = client.twitter
stats_coll = db.twitter_statuses
retweets_coll = db.twitter_retweets
favs_coll = db.twitter_favorites
lists_coll = db.twitter_lists_updated
friends_coll = db.twitter_friends_updated


In [97]:
import psycopg2
import psycopg2.extras
import pandas as pd

# Connect to the database
connection = psycopg2.connect(host='3.230.203.12',
                             user='compdb',
                             port=5438,
                             database='twitter',
                             password='compdbs_postgres')
connection.set_session(readonly=True, autocommit=True)

cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

#Work

##Tweets per weekday

In [98]:
x = 1
weekdayCountList=[]
total = 0
ototal = 0
while x < 8:
    query = "SELECT count(distinct(statuses)) FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    weekdayCountList.append([x,res[0]['count']])

    x=x+1


print(weekdayCountList)


[[1, 6618], [2, 6264], [3, 6684], [4, 6647], [5, 6729], [6, 5172], [7, 4717]]


In [99]:
import plotly.express as px
fig = px.bar(weekdayCountList, x=0, y=1, title = "Tweets by weekday",labels={'0':'Day of Week','1':'Number of Tweets'}, range_y=(4000,7000))
fig.show()

##Retweets and Likes per weekday

In [100]:
x = 1
weekDayTotals=[]
total = 0
ototal = 0
while x < 8:
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for i in res:
        total = total+i['favorites_count']

    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for w in res:
        ototal = ototal+w['retweet_count']

    weekDayTotals.append([x,total,ototal])
    x=x+1
    ototal=0
    total=0


print(weekDayTotals)

[[1, 18805170, 8095532], [2, 20967480, 8806093], [3, 22649066, 11023208], [4, 21073340, 11227533], [5, 28259773, 11119761], [6, 20573004, 16762835], [7, 22489203, 8907312]]


In [101]:
retweetday = []
likesday = []
x=1
for i in weekDayTotals:
    retweetday.append([x,i[2]])
    likesday.append([x,i[1]])

    x=x+1

In [102]:
fig = px.bar(retweetday, x=0, y=1, title = "retweets by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,20000000))
fig.show()

In [103]:
fig = px.bar(likesday, x=0, y=1, title = "Likes by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,30000000))
fig.show()

##Retweets per Tweet and Likes per Tweet per weekday

In [104]:
x = 1
avgRetweetpd = []
avgLikespd = []
while x < 8:
    average = (weekDayTotals[x-1][2])/(weekdayCountList[x-1][1])
    avgRetweetpd.append([x,average])

    average = (weekDayTotals[x-1][1])/(weekdayCountList[x-1][1])
    avgLikespd.append([x,average])
    x=x+1

print(avgRetweetpd)
print(avgLikespd)


[[1, 1223.25959504382], [2, 1405.8258301404853], [3, 1649.1932974266906], [4, 1689.112832856928], [5, 1652.513152028533], [6, 3241.074052590874], [7, 1888.3425906296375]]
[[1, 2841.5185856754306], [2, 3347.2988505747126], [3, 3388.549670855775], [4, 3170.3535429517074], [5, 4199.698766532917], [6, 3977.7656612529004], [7, 4767.691965232139]]


Retweet per tweet

In [105]:
fig = px.bar(avgRetweetpd, x=0, y=1, title = "retweet per tweet by weekday",labels={'0':'Day of Week','1':'Retweets per Tweet'}, range_y=(0,4000))
fig.show()

Like Per Tweet

In [106]:
fig = px.bar(avgLikespd, x=0, y=1, title = "like per tweet by weekday",labels={'0':'Day of Week','1':'Like per Tweet'}, range_y=(0,5000))
fig.show()

##Tweets per Week of Month

In [107]:
tweetsPerWeek = []
x=0
y=7
i=1
while y<36:
    query = "SELECT count(distinct(status_id)) FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()
    tweetsPerWeek.append([i,res[0]['count']])
    x=y+1
    if(y==21):
        y=y+10
    else:
        y=y+7
    i=i+1

print(tweetsPerWeek)



[[1, 10374], [2, 11578], [3, 9234], [4, 11645]]


In [108]:
fig = px.bar(tweetsPerWeek, x=0, y=1, title = "Tweets by week",labels={'0':'Week of Month','1':'Number of Tweets'}, range_y=(5000,12000))
fig.show()

##Retweets and Likes per week of Month

In [109]:
totalsPerWeek = []
total = 0
ototal = 0
x=0
y=7
num=1
while y<36:
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for counts in res:
        total = total+counts['favorites_count']


    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for retweets in res:
        ototal = ototal+retweets['retweet_count']

    totalsPerWeek.append([num,total,ototal])
    x=y+1
    if(y==21):
        y=y+10
    else:
        y=y+7
    num=num+1
    ototal=0
    total=0

print(totalsPerWeek)

[[1, 36996688, 25150022], [2, 34127560, 16885833], [3, 35106722, 13797065], [4, 48586066, 20109354]]


In [110]:
retweetweek = []
likesweek = []
x=1
for i in totalsPerWeek:
    retweetweek.append([x,i[2]])
    likesweek.append([x,i[1]])

    x=x+1

In [111]:
fig = px.bar(retweetweek, x=0, y=1, title = "retweets by week of month",labels={'0':'Week of Month','1':'Retweet per Tweet'}, range_y=(0,30000000))
fig.show()

In [112]:
fig = px.bar(likesweek, x=0, y=1, title = "Likes by week of month",labels={'0':'Week of Month','1':'Like per Tweet'}, range_y=(0,50000000))
fig.show()

##Retweets per Tweet and Likes per Tweet per Week of Month

In [113]:
print(totalsPerWeek)
print(tweetsPerWeek)

[[1, 36996688, 25150022], [2, 34127560, 16885833], [3, 35106722, 13797065], [4, 48586066, 20109354]]
[[1, 10374], [2, 11578], [3, 9234], [4, 11645]]


In [114]:
x = 1
avgRetweetpw = []
avgLikespw = []
while x < 5:
    average = (totalsPerWeek[x-1][2])/(tweetsPerWeek[x-1][1])
    avgRetweetpw.append([x,average])

    average = (totalsPerWeek[x-1][1])/(tweetsPerWeek[x-1][1])
    avgLikespw.append([x,average])
    x=x+1

print(avgRetweetpw)
print(avgLikespw)

[[1, 2424.3321765953347], [2, 1458.4412679219208], [3, 1494.1590859865714], [4, 1726.8659510519535]]
[[1, 3566.2895700790436], [2, 2947.621350837796], [3, 3801.8975525232836], [4, 4172.268441391155]]


In [115]:
fig = px.bar(avgRetweetpw, x=0, y=1, title = "retweet per tweet by week of month",labels={'0':'week of month','1':'Retweets per Tweet'}, range_y=(0,4000))
fig.show()

In [116]:
fig = px.bar(avgLikespw, x=0, y=1, title = "like per tweet by week of month",labels={'0':'week of month','1':'Like per Tweet'}, range_y=(0,5000))
fig.show()

##Other Exploration

In [117]:
query = "SELECT created_date,count(distinct(statuses)) FROM twitter.statuses GROUP BY created_date"
cursor.execute(query) #Executing query
res = cursor.fetchall()
max = 0
for i in res:
    if i['count']>max:
        max=i['count']
        print(max)
        newres = i
print(newres)




2
3
10
17
19
20
24
26
27
29
32
34
36
49
67
71
74
83
123
126
128
157
161
175
179
180
205
220
259
262
305
315
353
403
417
435
514
526
674
781
893
1022
RealDictRow([('created_date', datetime.date(2021, 1, 11)), ('count', 1022)])


In [118]:
query = "SELECT created_date,sum(favorites_count) FROM twitter.statuses GROUP BY created_date"
cursor.execute(query) #Executing query
res = cursor.fetchall()
print(res)
max = 0
for i in res:
    if i['sum']>max:
        max=i['sum']
        print(max)
        newres = i
print(newres)

[RealDictRow([('created_date', datetime.date(2019, 10, 15)), ('sum', 26920)]), RealDictRow([('created_date', datetime.date(2015, 6, 13)), ('sum', 5)]), RealDictRow([('created_date', datetime.date(2019, 9, 8)), ('sum', 27459)]), RealDictRow([('created_date', datetime.date(2016, 8, 6)), ('sum', 3)]), RealDictRow([('created_date', datetime.date(2020, 6, 14)), ('sum', 80783)]), RealDictRow([('created_date', datetime.date(2016, 9, 27)), ('sum', 604)]), RealDictRow([('created_date', datetime.date(2019, 2, 20)), ('sum', 14040)]), RealDictRow([('created_date', datetime.date(2018, 11, 5)), ('sum', 1241)]), RealDictRow([('created_date', datetime.date(2019, 2, 7)), ('sum', 1480)]), RealDictRow([('created_date', datetime.date(2017, 8, 9)), ('sum', 55)]), RealDictRow([('created_date', datetime.date(2015, 12, 31)), ('sum', 180)]), RealDictRow([('created_date', datetime.date(2018, 11, 28)), ('sum', 4780)]), RealDictRow([('created_date', datetime.date(2015, 9, 8)), ('sum', 133)]), RealDictRow([('creat

#T-Testing

##Per Weekday

In [161]:
import numpy as np
from scipy import stats
import re
sum = 0
x = 1
totalOther = []
totalDay = []
while x < 8:
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for i in res:
        if(x!=1):
            totalOther.append(i['favorites_count'])
        else:
            totalDay.append(i['favorites_count'])
    x=x+1



totalDay2 = []
totalOther2 = []
x=1
sum2=0
while x < 8:
    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for i in res:
        if(x!=1):
            totalOther2.append(i['retweet_count'])
        else:
            totalDay2.append(i['retweet_count'])
    x=x+1



print(str(stats.ttest_ind(totalOther, totalDay)))
print(str(stats.ttest_ind(totalOther2, totalDay2))) #running a t-test
#If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null
#hypothesis of identical average scores. The null hypothesis here is that the retweet counts averages are the same
#If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%,
#then we reject the null hypothesis of equal averages.
#Therefore even though one of our sub sample groups had a p-value below 0.05 or 0.1, all other groups were above
#Having as high as 37.9%. Therefore we cannot reject the null hypthesis: meaning we cannot reject
#that the average retweet count between the two groups are the same.

TtestResult(statistic=2.6647033197546555, pvalue=0.007708514095540831, df=42829.0)
TtestResult(statistic=1.9098304005343465, pvalue=0.05616172668248298, df=42829.0)


In [160]:
import numpy as np
from scipy import stats
import re
sum = 0
x = 1
y=7
totalOther = []
totalDay = []
totalDay2 = []
totalOther2 = []
resultsLikes=[]
resultsRetweets=[]
while y>0:
    while x < 8:
        query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
        cursor.execute(query) #Executing query
        res = cursor.fetchall()

        for i in res:
            if(x!=y):
                totalOther.append(i['favorites_count'])
            else:
                totalDay.append(i['favorites_count'])

        query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '%d'" % x
        cursor.execute(query) #Executing query
        res = cursor.fetchall()

        for i in res:
            if(x!=y):
                totalOther2.append(i['retweet_count'])
            else:
                totalDay2.append(i['retweet_count'])
        x=x+1
    resultsLikes.append(str(stats.ttest_ind(totalOther, totalDay)))
    resultsRetweets.append(str(stats.ttest_ind(totalOther2, totalDay2)))
    y=y-1



print(resultsLikes)






#running a t-test
#If we observe a large p-value, for example larger than 0.05 or 0.1, then we cannot reject the null
#hypothesis of identical average scores. The null hypothesis here is that the retweet counts averages are the same
#If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%,
#then we reject the null hypothesis of equal averages.
#Therefore even though one of our sub sample groups had a p-value below 0.05 or 0.1, all other groups were above
#Having as high as 37.9%. Therefore we cannot reject the null hypthesis: meaning we cannot reject
#that the average retweet count between the two groups are the same.

['TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)', 'TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)', 'TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)', 'TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)', 'TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)', 'TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)', 'TtestResult(statistic=-3.2708685675096336, pvalue=0.0010730242586626533, df=42829.0)']


##Per Week of Month

In [126]:
mtotalOther = []
mtotalWeek = []
mtotalOther2 = []
mtotalWeek2 = []
x=0
y=7
sum=0
while y<36:
    query = "SELECT distinct(status_id),favorites_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for counts in res:
        if(x!=22):
            mtotalOther.append(counts['favorites_count'])
        else:
            mtotalWeek.append(counts['favorites_count'])



    query = "SELECT distinct(status_id),retweet_count FROM twitter.statuses WHERE extract(day FROM created_date) BETWEEN '%d' and '%d'" % (x,y)
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    for num in res:
        if(x!=22):
            mtotalOther2.append(num['retweet_count'])
        else:
            mtotalWeek2.append(num['retweet_count'])
            sum = sum + num['retweet_count']

    #totalsPerWeek.append([num,total,ototal])
    x=y+1
    if(y==21):
        y=y+10
    else:
        y=y+7


print(str(stats.ttest_ind(mtotalOther, mtotalWeek)))
print(str(stats.ttest_ind(mtotalOther2, mtotalWeek2)))


TtestResult(statistic=-2.7476253697687216, pvalue=0.006005365820634144, df=42829.0)
TtestResult(statistic=0.2293955109568773, pvalue=0.8185626202846767, df=42829.0)


#When does Elon Musk Tweet?

In [86]:
x = 1
weekdayCountElon=[]
totalElon = 0
ototalElon = 0
while x < 8:
    query = "SELECT count(distinct(statuses)) FROM twitter.statuses WHERE user_id = '44196397' AND extract(isodow FROM created_date) = '%d'" % x
    cursor.execute(query) #Executing query
    res = cursor.fetchall()

    weekdayCountElon.append([x,res[0]['count']])

    x=x+1


print(weekdayCountList)
import plotly.express as px
fig = px.bar(weekdayCountElon, x=0, y=1, title = "Tweets by weekday",labels={'0':'Day of Week','1':'Number of Tweets'})
fig.show()


[[1, 76], [2, 70], [3, 77], [4, 98], [5, 72], [6, 81], [7, 97]]


#Putting It All Together

In [165]:
query = "SELECT favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '5' AND extract(day FROM created_date) BETWEEN '22' and '32'"
cursor.execute(query) #Executing query
res = cursor.fetchall()
bestTweets = []
for x in res:
    bestTweets.append(x['favorites_count'])


query = "SELECT favorites_count FROM twitter.statuses WHERE extract(isodow FROM created_date) != '5' AND extract(day FROM created_date) NOT BETWEEN '22' and '32'"
cursor.execute(query) #Executing query
res = cursor.fetchall()
otherTweets = []
for x in res:
    otherTweets.append(x['favorites_count'])

print(str(stats.ttest_ind(otherTweets, bestTweets)))

TtestResult(statistic=-4.398376407348794, pvalue=1.0947066550226364e-05, df=27659.0)


In [163]:
query = "SELECT retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) = '6' AND extract(day FROM created_date) BETWEEN '1' and '7'"
cursor.execute(query) #Executing query
res = cursor.fetchall()
bestTweets = []
for x in res:
    bestTweets.append(x['retweet_count'])


query = "SELECT retweet_count FROM twitter.statuses WHERE extract(isodow FROM created_date) != '6' AND extract(day FROM created_date) NOT BETWEEN '1' and '7'"
cursor.execute(query) #Executing query
res = cursor.fetchall()
otherTweets = []
for x in res:
    otherTweets.append(x['retweet_count'])

print(str(stats.ttest_ind(otherTweets, bestTweets)))

TtestResult(statistic=-5.168866635358705, pvalue=2.3703694891158435e-07, df=29787.0)
